In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np
import os
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict
from PIL import Image,ImageFile
import glob
from shutil import copy
import time
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# Load the Data
data_dir = 'CG_2'
html_dir = '/datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/'
# Define a transform to normalize the data
test_transforms = transforms.Compose([transforms.Resize((224,224)),                                       
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                                       
                             ])

In [3]:
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

## Run 1

out_dir  = 'evaluation/first_run/1'
#Load Model
model = models.densenet121(pretrained=True)
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 10)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))

model.classifier = classifier
state_dict = torch.load('/datadrive2/amit_cvnd/deep-learning-pytorch/models.first.run/cg_e2_lr_001_loss_1_acc_71_checkpoint.pth')
model.load_state_dict(state_dict)

## Run 2
out_dir  = 'evaluation/first_run/2_html_images'
#Load Model
model = models.densenet161(pretrained=True)
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(2208, 1000)),
                          ('relu', nn.ReLU()),
                          ('dropout1',nn.Dropout(p=0.4)),
                          ('fc2', nn.Linear(1000, 10)),                          
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
model.classifier = classifier
state_dict = torch.load('/datadrive2/amit_cvnd/deep-learning-pytorch/models.first.run/cg_no_dropout_e5_lr_0005_loss_1_acc_80_checkpoint.pth')
model.load_state_dict(state_dict)

In [4]:
## PF RUN 3
out_dir  = 'evaluation/pf_run_3/1_html_images'
#Load Model
model = models.vgg19_bn(pretrained=True)
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, 4096)),
                          ('relu1', nn.ReLU()),
                          ('dropout1',nn.Dropout(p=0.5)),
                          ('fc2', nn.Linear(4096, 1000)), 
                          ('relu2', nn.ReLU()),
                          ('dropout2',nn.Dropout(p=0.5)),
                          ('fc3', nn.Linear(1000, 5)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
model.classifier = classifier
state_dict = torch.load('/datadrive2/amit_cvnd/deep-learning-pytorch/models/pf_run_3/cg_e4_lr_0.0008_loss_0.242_acc_0.911_config1_checkpoint.pth')
model.load_state_dict(state_dict)

In [5]:
def view_classify(img, ps, version="MNIST"):
    ''' Function for viewing an image and it's predicted classes.
    '''
    ps = ps.data.numpy().squeeze()

    fig, ( ax1,ax2) = plt.subplots(figsize=(18,27), ncols=2)
    ax1.imshow(img.resize_(1, 224, 224).numpy().squeeze())
    ax1.axis('off')
   
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    ax2.set_yticklabels(['brochure',
                            'charts',
                            'fund_manager_no_keypoints',
                            'fund_managers',
                            'junk',
                            'logos',
                            'logos_will',
                            'no_keypoints',
                            'other_logo',
                            'others'], size='small');
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()

In [6]:
def find_classes(dir):
    classes = os.listdir(dir)
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx

In [7]:
classes ,_ = find_classes(dir=data_dir + '/test')
failed_image = 0 
# Test out your network!
model.eval()
start = time.time()
# For testing Training & Evaluation data
# path = data_dir +'/test'

# For HTML Data
path = html_dir
files = glob.glob(path + '/**/*.jp*g', recursive=True)
print("Images to classify : ",len(files))
for image_path in files:
    #print(image_path)
    try:
        image = pil_loader(image_path)
    except :
        print("Image did not open : {}".format(image_path))
        failed_image +=1
        continue
    image = test_transforms(image)
    image.unsqueeze_(0) 
    # Calculate the class probabilities (softmax) for img
    with torch.no_grad():
        output = model.forward(image)
    ps = torch.exp(output)
    #print(classes[ps.data.numpy().argmax()])
    pred = classes[ps.data.numpy().argmax()]
    # Create Path to save the model
    output_path=os.path.join(out_dir,pred)
    if not os.path.exists(output_path):
        os.makedirs(output_path)    
    copy(image_path,output_path)    
    
    # Plot the image and probabilities
    #view_classify(image, ps)
    #break
end = time.time()
print("Failed Images count : {}".format(failed_image))
print("Time Taken : {}".format(end-start))

Images to classify :  146124
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/17540.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/15333.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/113648.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/78772.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/1714.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/38293.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/121584.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/78084.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/117511.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/77248.jpeg
Image did not open : /datadrive2/amit_c

Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/141145.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/142213.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/5523.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/12915.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/117791.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/87540.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/132596.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/139309.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/40364.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/88065.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/do

/opt/rh/rh-python35/root/usr/lib64/python3.5/site-packages/PIL/Image.py:953: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/115071.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/15145.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/10602.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/112690.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/22167.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/40301.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/22942.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/118869.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/50372.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/18773.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/dow

Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/17654.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/47092.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/67154.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/100959.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/168.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/50338.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/131369.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/14901.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/35964.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/23365.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downla

...

Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/8276.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/18182.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/104781.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/17260.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/51858.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/17521.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/112631.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/20678.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/4967.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/90358.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downla

Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/129430.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/5330.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/62955.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/142317.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/82820.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/128667.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/12749.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/122145.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/129920.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/75271.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/do

Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/4648.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/70395.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/144231.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/73739.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/4577.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/84441.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/333.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/14264.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/51986.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/17936.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaode

Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/20080.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/79994.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/70570.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/13190.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/112464.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/53464.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/66432.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/64265.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/129312.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/144129.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/dow

Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/79079.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/60426.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/25616.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/131780.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/81681.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/102038.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/52352.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/128249.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/5153.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/downlaoded_images/88691.jpeg
Image did not open : /datadrive2/amit_cvnd/ComputerVisionDegree/down